In [ ]:
import torch
import plotly.graph_objects as go
import numpy as np

from _src import ModelConfig
from _src import GaussianProcess, tStudentProcess

In [5]:
# Define the true function
def true_function(X):
    return torch.sin(X).squeeze()

def generate_synthetic_data(n_points=20, noise_level=0.2, n_outliers=0, outlier_magnitude=3.0, func=true_function):
    """
    Generates synthetic data based on a given true function with added Gaussian noise,
    and optionally inserts outliers.

    Parameters:
    - n_points: Number of training data points.
    - noise_level: Standard deviation of the Gaussian noise.
    - n_outliers: Number of outliers to insert.
    - outlier_magnitude: Magnitude multiplier for the outliers.
    - func: True function to generate the underlying data.

    Returns:
    - X_train: Training input tensor.
    - y_train: Training output tensor with noise and outliers.
    """
    torch.manual_seed(42)
    X_train = torch.linspace(0, 10, n_points).unsqueeze(1)
    y_train = func(X_train) + noise_level * torch.randn(X_train.size(0))
    
    # Insert outliers
    if n_outliers > 0:
        outlier_indices = torch.randperm(n_points)[:n_outliers]
        y_train[outlier_indices] += outlier_magnitude * torch.randn(n_outliers)
    
    return X_train, y_train

def initialize_and_fit_models(X_train, y_train, noise_level=0.1, df=2):
    config_gp = ModelConfig(ard_num_dims=1, kernel_type="matern32", trained_params={"lengthscale": 1.0, "outputscale": 1.0, "noise": noise_level})
    config_tsp = ModelConfig(ard_num_dims=1, kernel_type="matern32", trained_params={"lengthscale": 1.0, "outputscale": 1.0, "noise": noise_level, "df": df})

    gp = GaussianProcess(config_gp, optimize=True, mprior=0.0)
    tsp = tStudentProcess(config_tsp, optimize=True, mprior=0.0)

    n_steps = 1000 * 3
    lr = 0.01

    gp.fit(X_train, y_train, n_steps=n_steps, lr=lr)
    tsp.fit(X_train, y_train, n_steps=n_steps)

    return gp, tsp

def plot_predictions(gp, tsp, X_train, y_train, X_test, func=true_function):
    """
    Generates a plot for the predictions, confidence intervals of both models, 
    and the true function trajectory.

    Parameters:
    - gp: Fitted GaussianProcess model.
    - tsp: Fitted tStudentProcess model.
    - X_train: Training input tensor.
    - y_train: Training output tensor.
    - X_test: Test input tensor for predictions.
    - func: True function to plot for reference.
    """
    fmean_gp, fcov_gp = gp.predict(X_test, return_std=True)
    fmean_tsp, fcov_tsp = tsp.predict(X_test, return_std=True)

    # Convert predictions to numpy for plotting
    fmean_gp = fmean_gp.detach().numpy()
    std_dev_gp = fcov_gp.detach().numpy() if isinstance(fcov_gp, torch.Tensor) else np.sqrt(np.diag(fcov_gp))
    fmean_tsp = fmean_tsp.detach().numpy()
    std_dev_tsp = fcov_tsp.detach().numpy() if isinstance(fcov_tsp, torch.Tensor) else np.sqrt(np.diag(fcov_tsp))

    # Calculate the true function values
    true_y = func(X_test).numpy()

    fig = go.Figure()

    # Training data
    fig.add_trace(go.Scatter(x=X_train.numpy().flatten(), y=y_train.numpy(), mode='markers', name='Training Data'))

    # True function trajectory
    fig.add_trace(go.Scatter(x=X_test.numpy().flatten(), y=true_y, mode='lines', name='True Function', line=dict(color='red')))

    # GP predictive mean
    fig.add_trace(go.Scatter(x=X_test.numpy().flatten(), y=fmean_gp, mode='lines', name='GP Predictive Mean'))

    # GP confidence interval
    fig.add_trace(go.Scatter(
        x=np.concatenate([X_test.numpy().flatten(), X_test.numpy().flatten()[::-1]]),
        y=np.concatenate([fmean_gp - 1.96 * std_dev_gp, (fmean_gp + 1.96 * std_dev_gp)[::-1]]),
        fill='toself',
        fillcolor='rgba(255,182,193,0.5)',
        line=dict(color='rgba(255,182,193,0)'), 
        showlegend=True,
        name='GP 95% Confidence Interval'
    ))

    # tSP predictive mean
    fig.add_trace(go.Scatter(x=X_test.numpy().flatten(), y=fmean_tsp, mode='lines', name='tSP Predictive Mean', line=dict(color='blue')))

    # tSP confidence interval
    fig.add_trace(go.Scatter(
        x=np.concatenate([X_test.numpy().flatten(), X_test.numpy().flatten()[::-1]]),
        y=np.concatenate([fmean_tsp - 1.96 * std_dev_tsp, (fmean_tsp + 1.96 * std_dev_tsp)[::-1]]),
        fill='toself',
        fillcolor='rgba(173,216,230,0.5)',
        line=dict(color='rgba(173,216,230,0)'),
        showlegend=True,
        name='tSP 95% Confidence Interval'
    ))

    # Final layout adjustments
    fig.update_layout(
        title="Gaussian Process vs. t-Student Process Regression",
        xaxis_title="X",
        yaxis_title="y",
        legend=dict(x=0, y=1.15, orientation="h"),
    )

    fig.show()

# Example usage of the functions
X_train, y_train = generate_synthetic_data(noise_level=0.2, n_outliers=1, outlier_magnitude=0.3, func=true_function)
gp, tsp = initialize_and_fit_models(X_train, y_train, noise_level=0.1, df=3.0)
X_test = torch.linspace(0, 10, 100).unsqueeze(1)
plot_predictions(gp, tsp, X_train, y_train, X_test, func=true_function)

Iter: 0, Log Marginal Likelihood: -23.77510643005371
Iter: 1, Log Marginal Likelihood: -23.70663070678711
Iter: 2, Log Marginal Likelihood: -23.638206481933594
Iter: 3, Log Marginal Likelihood: -23.569835662841797
Iter: 4, Log Marginal Likelihood: -23.50152015686035
Iter: 5, Log Marginal Likelihood: -23.433265686035156
Iter: 6, Log Marginal Likelihood: -23.36507797241211
Iter: 7, Log Marginal Likelihood: -23.296958923339844
Iter: 8, Log Marginal Likelihood: -23.228914260864258
Iter: 9, Log Marginal Likelihood: -23.160947799682617
Iter: 10, Log Marginal Likelihood: -23.093063354492188
Iter: 11, Log Marginal Likelihood: -23.02526092529297
Iter: 12, Log Marginal Likelihood: -22.957551956176758
Iter: 13, Log Marginal Likelihood: -22.889938354492188
Iter: 14, Log Marginal Likelihood: -22.822425842285156
Iter: 15, Log Marginal Likelihood: -22.75501251220703
Iter: 16, Log Marginal Likelihood: -22.687707901000977
Iter: 17, Log Marginal Likelihood: -22.620515823364258
Iter: 18, Log Marginal Lik

In [8]:
# Define the true function
def true_function(X):
    func = torch.sin(X) + X ** 2
    return func.squeeze()

# Example usage of the functions
X_train, y_train = generate_synthetic_data(noise_level=0.2, n_outliers=1, outlier_magnitude=1, func=true_function)
gp, tsp = initialize_and_fit_models(X_train, y_train, noise_level=0.1, df=3.0)
X_test = torch.linspace(0, 10, 100).unsqueeze(1)
plot_predictions(gp, tsp, X_train, y_train, X_test, func=true_function)

Iter: 0, Log Marginal Likelihood: -5302.21240234375
Iter: 1, Log Marginal Likelihood: -5250.07568359375
Iter: 2, Log Marginal Likelihood: -5198.6455078125
Iter: 3, Log Marginal Likelihood: -5147.92626953125
Iter: 4, Log Marginal Likelihood: -5097.9130859375
Iter: 5, Log Marginal Likelihood: -5048.609375
Iter: 6, Log Marginal Likelihood: -5000.01611328125
Iter: 7, Log Marginal Likelihood: -4952.12841796875
Iter: 8, Log Marginal Likelihood: -4904.94970703125
Iter: 9, Log Marginal Likelihood: -4858.47314453125
Iter: 10, Log Marginal Likelihood: -4812.69775390625
Iter: 11, Log Marginal Likelihood: -4767.623046875
Iter: 12, Log Marginal Likelihood: -4723.244140625
Iter: 13, Log Marginal Likelihood: -4679.552734375
Iter: 14, Log Marginal Likelihood: -4636.5478515625
Iter: 15, Log Marginal Likelihood: -4594.224609375
Iter: 16, Log Marginal Likelihood: -4552.5751953125
Iter: 17, Log Marginal Likelihood: -4511.5947265625
Iter: 18, Log Marginal Likelihood: -4471.275390625
Iter: 19, Log Marginal 

In [15]:
# Define the true function
def true_function(X):
    func = 5 * torch.sin(3*X) - X
    return func.squeeze()

# Example usage of the functions
X_train, y_train = generate_synthetic_data(noise_level=0.2, n_outliers=3, outlier_magnitude=5, func=true_function)
gp, tsp = initialize_and_fit_models(X_train, y_train, noise_level=0.1, df=3.0)
X_test = torch.linspace(0, 10, 100).unsqueeze(1)
plot_predictions(gp, tsp, X_train, y_train, X_test, func=true_function)

Iter: 0, Log Marginal Likelihood: -341.5642395019531
Iter: 1, Log Marginal Likelihood: -339.2976989746094
Iter: 2, Log Marginal Likelihood: -337.054931640625
Iter: 3, Log Marginal Likelihood: -334.83599853515625
Iter: 4, Log Marginal Likelihood: -332.6414489746094
Iter: 5, Log Marginal Likelihood: -330.47113037109375
Iter: 6, Log Marginal Likelihood: -328.32537841796875
Iter: 7, Log Marginal Likelihood: -326.2042541503906
Iter: 8, Log Marginal Likelihood: -324.1078186035156
Iter: 9, Log Marginal Likelihood: -322.0362548828125
Iter: 10, Log Marginal Likelihood: -319.9897155761719
Iter: 11, Log Marginal Likelihood: -317.968017578125
Iter: 12, Log Marginal Likelihood: -315.9714660644531
Iter: 13, Log Marginal Likelihood: -313.9999084472656
Iter: 14, Log Marginal Likelihood: -312.05352783203125
Iter: 15, Log Marginal Likelihood: -310.1320495605469
Iter: 16, Log Marginal Likelihood: -308.2356872558594
Iter: 17, Log Marginal Likelihood: -306.36407470703125
Iter: 18, Log Marginal Likelihood: 